## Проверка чанков **БЕЗ** сбора нового файла FAISS **БЕЗ** обращения к OpenAI


In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken


In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# Загружаем векторный файл базы знаний! В коллаб надо в папку faiss загрузить 2 файла

knowledge_only_langchain = FAISS.load_local("faiss", OpenAIEmbeddings())

In [ ]:
def answer_index(topic, top_similar_documents=10):

        docs = knowledge_only_langchain.similarity_search_with_score(topic, k=top_similar_documents)
        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f'\n=====================Отрывок документа №{i + 1}=====================\n{content}\n'
                print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                print(f' \n{content}\n')
                responses.append(response)

        return


In [ ]:
def run_dialog():

    while True:
        print('\n                 ====^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^= НОВЫЙ ВОПРОС =^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^==^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=\n\n')
        user_question = input('\nВопрос: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer_index(user_question)

    return

In [ ]:

run_dialog()


                 ====^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^= НОВЫЙ ВОПРОС =^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^==^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=^=



Вопрос: "Автомобиль серато в наличии белого цвета? И цена?"

=====================Отрывок документа №1=====================

=== score = 0.2319323718547821  Metadata документа ------------ {'Header 1': 'sales_purchases', 'Header 2': 'sales_purchases355'}
 
Обработанная запись базы знаний: Автомобиль Селтос белого цвета, стайл комплектация, полный привод 1,6, автомат доступен в городах: Волгоград, Магнитогорск, Оренбург,
Рязань. Цены на сайте носят информационный характер и могут отличаться от действительных цен дилеров Kia. Для бронирования автомобиля необходимо внести предоплату на
сайте www.kia.ru, автомобили для онлайн-бронирования помечены желтым значком. Предоплата может быть засчитана в счет стоимости автомобиля при заключении договора
купли-продажи.


=====================Отрывок документа №2=====================

=== score = 0.2

# Собрать новую базу FAISS

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re

def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())
    vectordateBase.save_local("faiss")

    return



In [ ]:
load_file_knowledge('database.md')
# БД сохранитсья в коллабе в папке faiss нужно подождать какое-то время